In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from proj1_helpers import *
#from implementations import *
DATA_FOLDER = '../data/'

In [3]:
from split_data import split_data

In [4]:
y_train, x_train, id_train = load_csv_data(DATA_FOLDER + 'train.csv')
y_test, x_test, id_test = load_csv_data(DATA_FOLDER + 'test.csv')

In [5]:
#x_train.filter(lambda v: v==v, x)

In [6]:
y_train

array([ 1., -1., -1., ...,  1., -1., -1.])

In [7]:
np.shape(x_train)

(250000, 30)

In [8]:
id_train

array([100000, 100001, 100002, ..., 349997, 349998, 349999])

### Basic Least Squares

In [18]:
def compare_prediction(w_train, x, y):
    y_pred = predict_labels(w_train, x)
    return (y_pred == y).sum()/len(y_pred)

In [10]:
w_ls, loss_ls = least_squares(y_train, x_train)
np.shape(w_ls)

(30,)

In [11]:
compare_prediction(w_ls, x_train, y_train)

0.74432799999999999

### Stochastic Gradient Descent

In [12]:
np.shape(np.asarray([1]*30)[:,None].reshape(30, 1))

(30, 1)

In [13]:
np.shape(np.asarray([1]*30))

(30,)

In [14]:
initial_w = np.asarray([1]*30)
max_iters = 30
gamma = 0.7

In [15]:
w_sgd, loss_sgd = least_squares_SGD(y_train, x_train, initial_w, max_iters, gamma)

/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/loss.py:5: RuntimeWarning: overflow encountered in square
  return np.mean(e**2)/2.


In [16]:
compare_prediction(w_sgd, x_train, y_train)

0.62978400000000001

### Ridge Regression

In [17]:
from build_polynomial import build_poly

def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    # split data
    x_tr, y_tr, x_te, y_te = split_data(x, y, ratio, seed)
    
    tr_poly = build_poly(x_tr, degree)
    te_poly = build_poly(x_te, degree)
    print(np.shape(x_tr))
    print(np.shape(tr_poly))
    #####
    #tr_poly = np.sum(np.split(tr_poly, degree+1, axis = 1), axis=0)
    #te_poly = np.sum(np.split(te_poly, degree+1, axis = 1), axis=0)
    #####
    
    loss = 1000
    w = 0
    
    for lambda_ in lambdas:
        # ridge regression
        weight, l = ridge_regression(y_tr, tr_poly, lambda_)
        #weight = stack_w(weight, degree)
        mse_test = compute_loss(y_te, te_poly, weight)
        
        if(mse_test < loss):
            loss = mse_test
            w = weight
        
    return w, loss

In [18]:
# boucler sur les seeds pour avoir un meileur ? 
seed = 69
degree = 6
split_ratio = 0.9

w_r, loss_ridge = ridge_regression_demo(x_train, y_train, degree, split_ratio, seed)

(225000, 30)
(225000, 210)


In [19]:
np.shape(y_train)

(250000,)

In [20]:
np.shape(w_r)

(210,)

In [21]:
w_s = np.sum(np.split(w_r, degree + 1),axis=0)
np.shape(w_s)

(30,)

In [22]:
np.shape(build_poly(x_train, degree))

(250000, 210)

In [23]:
compare_prediction(w_r.reshape(len(w_r), 1), build_poly(x_train, degree), y_train.reshape(len(y_train), 1))

0.79982799999999998

### Logistic Regression

In [24]:
def ridge_regression2(y, tx, lamb):
    """implement ridge regression."""
    aI = lamb * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

def ridge_regression3(y, tx, lambda_):
    first_term = tx.T.dot(tx)
    left = first_term + lambda_ *np.identity(tx.shape[1])
    right = tx.T.dot(y)
    w = np.linalg.solve(left, right)
    loss = compute_loss2(y, tx, w)
    return w, loss;

def compute_loss2(y, tx, w):
    e = y - tx.dot(w)
    return np.mean(e**2)/2.

### Bias-variance

In [25]:
from implementations import least_squares
from split_data import split_data

def bias_variance_demo():
    """The entry."""
    # define parameters
    seeds = range(100)
    num_data = 10000
    ratio_train = 0.005
    degrees = range(1, 10)
    lambdas = np.logspace(-5, 0, 15)
    
    best_seed = 0
    best_degree = 0
    best_lambda = 0
    min_loss = 1000
    for index_seed, seed in enumerate(seeds):
        np.random.seed(seed)
    
        # split data with a specific seed
        x_tr, y_tr, x_te, y_te = split_data(x_train, y_train, ratio_train, seed)
        
        # bias_variance_decomposition
        for index_deg, degree in enumerate(degrees):
            # form data with polynomial degree
            train_poly = build_poly(x_tr, degree)
            test_poly = build_poly(x_te, degree)
            
            w_train, loss_train = least_squares(y_tr, train_poly)

            # calculate the loss for train and test data
            loss_test = compute_loss(y_te, test_poly, w_train)
            
            if(loss_test < min_loss):
                min_loss = loss_test
                best_seed = seed
                #best_degree = degree
                best_lambda = lambda_
    #return best_seed, best_lambda, min_loss
    return best_seed, best_degree ,min_loss        


In [26]:
#s, d, l = bias_variance_demo()

In [27]:
#l

### Cross-validation

In [28]:
from build_k_indices import build_k_indices
from implementations import ridge_regression
from build_polynomial import build_poly

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""

    # get k'th subgroup in test, others in train
    train_indices = [x for j, x in enumerate(k_indices) if j != k]
    train_indices = [idx for part in train_indices for idx in part]
    test_indices = k_indices[k]
    
    x_tr = x[train_indices]
    y_tr = y[train_indices]
    
    x_te = x[test_indices]
    y_te = y[test_indices]

    # form data with polynomial degree
    #train_poly = build_poly(x_tr, degree)
    #test_poly = build_poly(x_te, degree)

    # ridge regression
    
    weights_train, loss_tr = ridge_regression(y_tr, x_tr, lambda_)

    # calculate the loss for train and test data
    loss_te = compute_loss(y_te, x_te, weights_train)

    return loss_tr, loss_te

In [29]:
def cross_validation_demo():
    seed = 55
    degree = 2
    k_fold = 4
    lambdas = np.logspace(-4, 0, 30)
    
    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)
    
    min_mean_loss = 1000
    best_lambdas = np.zeros(k_fold)
    # cross validation
    for i_test in range(k_fold):
        min_loss = 1000
        for lambda_ in lambdas:
            loss_tr, loss_te = cross_validation(y_train, x_train, k_indices, i_test, lambda_, degree)
            
            if(loss_te < min_loss):
                min_loss = loss_te
                best_lambdas[i_test] = lambda_
    lambda_mean = np.mean(best_lambdas)
    return lambda_mean, min_mean_loss

In [30]:
bl, mml = cross_validation_demo()

In [31]:
bl

0.25007499999999999

In [32]:
w, l = ridge_regression(y_train, x_train, bl)

In [33]:
compare_prediction(w, x_train, y_train)

0.74440799999999996

### Logistic Regression

In [8]:
max_iters = 1
gamma = 0.5
lambda_ = 0.9

x_train2 = x_train[:2]
y_train2 = y_train.reshape(len(y_train), 1)
y_train2 = y_train2[:2]

print(np.shape(x_train2))
print(np.shape(y_train2))

tx = np.c_[np.ones((y_train2.shape[0], 1)), x_train2]
initial_w = np.zeros((tx.shape[1], 1))

w , loss = logistic_regression(y_train2, tx, initial_w, max_iters, gamma)
#w , loss = reg_logistic_regression(y_train2, tx, lambda_, initial_w, max_iters, gamma)

loss

(2, 30)
(2, 1)


1.3862943611198906

In [12]:
print(loss)
print(np.shape(x_train2))
print(np.shape(tx))
print(np.shape(w))
print(np.shape(y_train2))
w

1.38629436112
(2, 30)
(2, 31)
(31, 1)
(2, 1)


array([[ -1.00000000e-02],
       [ -1.72170500e+00],
       [ -7.73245000e-01],
       [ -1.05939000e+00],
       [ -5.82290000e-01],
       [  1.49895500e+01],
       [  1.56085550e+01],
       [  1.49983300e+01],
       [ -3.67750000e-02],
       [  1.78470000e-01],
       [ -8.88555000e-01],
       [ -5.27500000e-03],
       [ -1.42300000e-02],
       [  1.49860000e+01],
       [ -4.67020000e-01],
       [ -2.55000000e-02],
       [  4.70700000e-02],
       [ -2.95640000e-01],
       [  3.85000000e-03],
       [ -1.36150000e-02],
       [ -5.86440000e-01],
       [  2.73550000e-02],
       [ -1.17452500e+00],
       [ -5.00000000e-03],
       [ -3.56215000e-01],
       [ -1.25000000e-04],
       [ -1.51500000e-02],
       [  1.52153100e+01],
       [  1.49912000e+01],
       [  1.49726250e+01],
       [ -1.25905000e-01]])

In [ ]:
#marche pas ... a voir

#compare_prediction(w, tx, y_train)

In [16]:
def logistic_regression_demo():
    max_iters = 10
    #gamma = 0.01
    gammas = np.arange(0.01,0.5,0.01)
    
    x_train2 = x_train
    y_train2 = y_train.reshape(len(y_train), 1)
    
    tx = np.c_[np.ones((y_train2.shape[0], 1)), x_train2]
    w = np.zeros((tx.shape[1], 1))
    
    best_w = 0
    best_loss = 1000
    best_gamma = 0
    
    for gamma_ in gammas:
        
        w_log , loss = logistic_regression(y_train2, tx, w, max_iters, gamma_)
        
        #print(gamma_)
        print(loss)
        
        if(loss < best_loss ):
            best_loss = loss
            best_gamma = gamma_
            best_w = w_log

    
    return best_w, best_loss, best_gamma


In [17]:
# Problem
w, loss, gamma = logistic_regression_demo()

173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514
173286.79514


In [13]:
w

0

In [26]:
def sigmoid(t):
    """apply sigmoid function on t."""
    #return 2*(np.exp(t) / (1 + np.exp(t))) - 1
    return (np.exp(t) -1) / (np.exp(t)+1)

In [34]:
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    sig = sigmoid(tx.dot(w))
    loss = y.T @ (np.log(sig)) + (1 - y).T @ (np.log(1 - sig))
    return np.squeeze(- loss)

def calculate_loss2(y, tx, w):
    """compute the cost by negative log likelihood."""
    
    loss = np.log(1 + np.exp(tx @ w)) - (y * (tx @ w))
    
    sum_loss = np.sum(loss)
    
    return sum_loss


In [28]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    
    return  tx.T @(sigmoid(tx @ w) - y)

In [29]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y, tx, w)
    w = w - gamma * grad
    return loss, w

In [39]:
def logistic_regression_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 2
    threshold = 1e-8
    gamma = 0.01
    lambda_ = 0.9
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        #loss, w = learning_by_newton_method(y, tx, w)
        #loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return loss, w

nb = 20000

x_train2 = x_train[:nb]
y_train2 = y_train.reshape(len(y_train), 1)
y_train2 = y_train2[:nb]

loss, w = logistic_regression_gradient_descent_demo(y_train2, x_train2)
#logistic_regression_newton_method_demo(y, x)

/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.
/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [38]:
loss

13862.943611198905

In [30]:
def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    pred = sigmoid(tx.dot(w))
    pred = np.diag(pred.T[0])
    r = np.multiply(pred, (1-pred))
    return tx.T.dot(r).dot(tx)



In [31]:
def logistic_regression(y, tx, w):
    """return the loss, gradient, and hessian."""
    loss = calculate_loss(y, tx, w)
    gradient = calculate_gradient(y, tx, w)
    hessian = calculate_hessian(y, tx, w)
    return loss, gradient, hessian

In [11]:
def learning_by_newton_method(y, tx, w):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    loss, gradient, hessian = logistic_regression(y, tx, w)
    w -= np.linalg.solve(hessian, gradient)
    return loss, w

In [12]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    # return loss, gradient, and hessian
    loss = calculate_loss(y, tx, w) + (w.T@w*lambda_ / 2.0)
    
    grad = calculate_gradient(y, tx, w) + (w*lambda_)
    
    H = calculate_hessian(y, tx, w) + (w.T@w*lambda_ / 2.0)
    
    
    return loss, grad, H

In [13]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    # return loss, gradient
    loss, grad, H = penalized_logistic_regression(y, tx, w, lambda_)
    # update w
    
    w = w - gamma * np.linalg.solve(H, grad);
    
    
    return loss, w

In [24]:
def logistic_regression_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 1
    threshold = 1e-8
    gamma = 0.01
    lambda_ = 0.9
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        #loss, w = learning_by_newton_method(y, tx, w)
        #loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return loss, w

nb = 200

x_train2 = x_train[:nb]
y_train2 = y_train.reshape(len(y_train), 1)
y_train2 = y_train2[:nb]

loss, w = logistic_regression_gradient_descent_demo(y_train2, x_train2)
#logistic_regression_newton_method_demo(y, x)

/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [25]:
loss


array(nan)

In [38]:
w

array([[ -1.57085056e-01],
       [  1.65317219e-05],
       [ -1.33832950e-03],
       [ -1.28434808e-03],
       [ -1.44225246e-04],
       [ -3.55491485e-03],
       [  9.27900340e-05],
       [ -5.01911896e-03],
       [  7.00295081e-02],
       [ -1.39055623e-04],
       [ -3.42766892e-02],
       [ -4.12212280e-02],
       [  1.84676417e-02],
       [  1.27154137e-02],
       [  3.60760574e-02],
       [ -7.09694051e-04],
       [ -3.50438682e-04],
       [  3.68150579e-02],
       [  6.94074028e-05],
       [ -7.01398203e-04],
       [  7.34332123e-04],
       [  3.45887833e-04],
       [ -7.78494145e-05],
       [ -5.00872622e-02],
       [ -1.78822212e-04],
       [  1.70315202e-04],
       [  7.81221814e-05],
       [ -3.32517186e-04],
       [ -1.71595954e-03],
       [ -2.11455686e-03],
       [  3.43226511e-02]])

In [39]:

tx = np.c_[np.ones((y_train2.shape[0], 1)), x_train2]

compare_prediction(w, tx, y_train2)

0.68149999999999999